# RIPEMD-160
** Implementation of the RIPEMD-160bit Algorithm **

*... in Python*

This implementation of 160 bit RIPEMD is for demonstrattion purposes to understand and see the RIPEMD algorithm in action with all intermediate steps. The RIPEMD [Pseudocode form the developer's site](https://homes.esat.kuleuven.be/~bosselae/ripemd/rmd160.txt) is used as a reference.

### Define and Select Test Cases

In [1]:
test_case=[["","9c1185a5c5e9fc54612808977ee8f548b2258d31"],\
           ["a","0bdc9d2d256b3ee9daae347be6f4dc835a467ffe"],\
           ["abc","8eb208f7e05d987a9b044a8e98c6b087f15a0bfc"],\
           ["message digest","5d0689ef49d2fae572b881b123a85ffa21595f36"],\
           ["abcdefghijklmnopqrstuvwxyz","f71c27109c692c1b56bbdceb5b9d2865b3708dbc"],\
           ["abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq","12a053384a9c0c88e405a06c27dcf49ada62eb2b"],\
           ["ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789","b0e20b6e3116640286ed3a87a5713079b21f5189"]]
use_test_case = 6
##
message = test_case[use_test_case][0]
ref_hash = test_case[use_test_case][1]

### Step 1 - Append Padding Bits

The messsage to be hashed is padded to have a length equal to 8 bytes {64 bits} less than being a multiple of 64 bytes {512 bits}. The padding step is performed even if the message length is already of desired length. The padding bit string used is `1` followed by `0` - `100...000`

The message length is eventually 56 bytes {448 bits}, 120 bytes {960 bits}, 184 bytes {1472 bits}, 248 bytes {1984 bits} and so on.

In [2]:
message_len = len(message)
message_len_bits = message_len * 8
print("Message Length : " + str(message_len) + " bytes {" + str(message_len_bits) + " bits}")

Message Length : 62 bytes {496 bits}


In [3]:
# Encode string to bytes
message_b = message.encode('utf-8')

In [4]:
# Calculate padding length
padding_len=(56-message_len)%64
padding_len=64 if (padding_len==0) else padding_len
print("Padding Length : " + str(padding_len) + " bytes {" + str(padding_len * 8) + " bits}")

Padding Length : 58 bytes {464 bits}


In [5]:
# Display Padded Message, length and calculation.
message_mod448 = message_b + b'\x80' + b'\x00' * (padding_len-1)
print("Padded Message :\n"+str(message_mod448))
print("\nlength(paddedMessage)      : "+str(len(message_mod448))+" bytes {"+str(len(message_mod448*8))+" bits}\nlength(paddedMessage) % 64 : "+str(len(message_mod448)%64)+" bytes {"+str((len(message_mod448)%64) * 8)+" bits}" )

Padded Message :
b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

length(paddedMessage)      : 120 bytes {960 bits}
length(paddedMessage) % 64 : 56 bytes {448 bits}


### Step 2 - Append Length

The bit length of the original message is appened to this _64 bits short of %512 bit_ message. This bit length is appeneded as an 8 byte {64 bits} little endian integer.

So, a message of length 14 bytes (_try test case # 3_) would have a bit length of 112 bits and the appended 64 bit little endian bit length would be `0x7000000000000000` (as hex) or `b'p\x00\x00\x00\x00\x00\x00\x00'` (as a byte string). If the message length is $> 2^{64}$ bits, only the lower 64 bits are used for padding.

In [6]:
# Append Length
processed_message=message_mod448+(message_len_bits%2**64).to_bytes(8,byteorder='little')
print("LSB64(len(unPaddedMessage)) : "+str((message_len_bits%2**64).to_bytes(8,byteorder='little')))
print("length( paddedMessage | LSB64(len(unPaddedMessage)) ) : "+str(len(processed_message))+" bytes {"+str(len(processed_message)*8)+" bits}")
print("\nPadded Message | LSB64(len(unPaddedMessage)) :\n"+str(processed_message))

LSB64(len(unPaddedMessage)) : b'\xf0\x01\x00\x00\x00\x00\x00\x00'
length( paddedMessage | LSB64(len(unPaddedMessage)) ) : 128 bytes {1024 bits}

Padded Message | LSB64(len(unPaddedMessage)) :
b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0\x01\x00\x00\x00\x00\x00\x00'


### Step 3 - Initilize RIPMD Context

In [7]:
h0 = 0x67452301
h1 = 0xEFCDAB89
h2 = 0x98BADCFE
h3 = 0x10325476
h4 = 0xC3D2E1F0

### Step 4 - Process Message in 16-Word Blocks

In [8]:
# Auxulary function that take in 3x 32bit words and return 1x32bit word.

def F(j, X, Y, Z):
    if (0<=j & j<=15):
        return (X^Y^Z) & 0xFFFFFFFF
    if (16<=j & j<=31):
        return ((X&Y) | (Z&~X)) & 0xFFFFFFFF
    if (32<=j & j<=47):
        return ((~Y|X) ^ Z) & 0xFFFFFFFF
    if (48<=j & j<=63):
        return ((X&Z) | (Y&~Z)) & 0xFFFFFFFF
    if (64<=j & j<=79):
        return (X^(~Z|Y)) & 0xFFFFFFFF
    else:
        raise ValueError('j is not in the range 0<=j<=79 !')

In [9]:
# Rotate Left
def rotl(x,s):
    return ( (x<<s) | x>>(32-s)) & 0xFFFFFFFF

In [10]:
# Shift Table
rol_table = [11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8, 7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12, 11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5, 11, 12, 14, 15, 14, 15, 9, 8, 9, 14, 5, 6, 8, 6, 5, 12, 9, 15, 5, 11, 6, 8, 13, 12, 5, 12, 13, 14, 11, 8, 5, 6]
rol_table_c = [8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6, 9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11, 9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5, 15, 5, 8, 11, 14, 14, 6, 14, 6, 9, 12, 9, 12, 5, 15, 8, 8, 5, 12, 9, 12, 5, 14, 6, 8, 13, 6, 5, 15, 13, 11, 11]

# r table (to use a sub-string of the message)
r_table = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8, 3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12, 1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2, 4, 0, 5, 9, 7, 12, 2, 10, 14, 1, 3, 8, 11, 6, 15, 13]
r_table_c = [5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12, 6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2, 15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13, 8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14, 12, 15, 10, 4, 1, 5, 8, 7, 6, 2, 13, 14, 0, 3, 9, 11]

# K Table
k_table = [0x0]*16 + [0x5A827999]*16 + [0x6ED9EBA1]*16 + [0x8F1BBCDC]*16 + [0xA953FD4E]*16
k_table_c = [0x50A28BE6]*16 + [0x5C4DD124]*16 + [0x6D703EF3]*16 + [0x7A6D76E9]*16 + [0x0]*16

In [11]:

def bytereverse(num32):
    rev_byte=0;
    for i in range(0,20):
        #print(hex(num32)+" "+hex(rev_byte))
        rev_byte = rev_byte << 8
        
        low_order_byte = num32 & 0xFF
        rev_byte = rev_byte | low_order_byte
        
        num32 = num32 >> 8
    return rev_byte

def disp_context(h4,h3,h2,h1,h0):
    print("h4..h0 = {:8x} {:8x} {:8x} {:8x} {:8x}".format(h4,h3,h2,h1,h0))

In [12]:
# Loop though the various 512 bit blocks of a long message.
for i in range(0,len(processed_message),64):
    print("PROCESSING bytes "+str(i)+"..."+str(i+64))
    M  = processed_message[i:i+64]
    print("\nMessage chunk being processed :\n"+str(M)+" \n")
    disp_context(h4,h3,h2,h1,h0)
    [A, B, C, D, E] = [h0, h1, h2, h3, h4]
    [Ac, Bc, Cc, Dc, Ec] = [h0, h1, h2, h3, h4]
    for j in range (0,80):
        rj = r_table[j]
        rjc = r_table_c[j]
        
        X = M[rj*4:rj*4+4]
        Xc = M[rjc*4:rjc*4+4]
        
        X_int = int.from_bytes(X,byteorder='little')
        Xc_int = int.from_bytes(Xc,byteorder='little')
        print("M  : "+"{:8x}".format(X_int)+"\nMc : "+"{:8x}".format(Xc_int))
        
        
        Wr = (A + F(j, B, C, D) + X_int + k_table[j]) & 0xFFFFFFFF
        T = (rotl(Wr, rol_table[j]) + E ) & 0xFFFFFFFF
        [A, E, D, C, B] = [E, D, rotl(C,10), B, T]
        
        Wr = (Ac + F(79-j, Bc, Cc, Dc) + Xc_int + k_table_c[j]) & 0xFFFFFFFF
        T = (rotl(Wr, rol_table_c[j]) + Ec) & 0xFFFFFFFF
        [Ac, Ec, Dc, Cc, Bc] = [Ec, Dc, rotl(Cc,10), Bc, T]
        
        
        disp_context(h4,h3,h2,h1,h0)

    T = (h1 + C + Dc) & 0xFFFFFFFF
    h1 = (h2 + D + Ec) & 0xFFFFFFFF
    h2 = (h3 + E + Ac) & 0xFFFFFFFF
    h3 = (h4 + A + Bc) & 0xFFFFFFFF
    h4 = (h0 + B + Cc) & 0xFFFFFFFF
    h0 = T
    
    

PROCESSING bytes 0...64

Message chunk being processed :
b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789\x80\x00' 

h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 44434241
Mc : 58575655
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 48474645
Mc : 37363534
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 4c4b4a49
Mc : 66656463
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 504f4e4d
Mc : 44434241
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 54535251
Mc : 6e6d6c6b
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 58575655
Mc : 4c4b4a49
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 62615a59
Mc : 76757473
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 66656463
Mc : 54535251
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 6a696867
Mc : 33323130
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67452301
M  : 6e6d6c6b
Mc : 62615a59
h4..h0 = c3d2e1f0 10325476 98badcfe efcdab89 67

In [13]:
# Compute output hash from the MD buffers.
output_int = h4<<128 | h3<<96 | h2 <<64 | h1 << 32 | h0

# The MD4 hash starts with the lowest order byte of A ... highest order byte of D
print("OUTPUT      : "+hex(bytereverse(output_int)))
print("REF. Hash   : 0x"+test_case[use_test_case][1])

OUTPUT      : 0xb0e20b6e3116640286ed3a87a5713079b21f5189
REF. Hash   : 0xb0e20b6e3116640286ed3a87a5713079b21f5189


## Compare with Python's `hashlib`

In [14]:
import hashlib

In [15]:
# The RIPEMD in Python's hashlib of RIPMED-160
H = hashlib.new('ripemd')
H.update(message_b)
ripemdhash=H.hexdigest()
print("Hashlib RIPEMD : 0x"+ripemdhash)

Hashlib RIPEMD : 0xb0e20b6e3116640286ed3a87a5713079b21f5189


### References

1. [Pseudocode form the developer's site](https://homes.esat.kuleuven.be/~bosselae/ripemd/rmd128.txt)
2. [Wikipedia](https://en.wikipedia.org/wiki/RIPEMD)
3. [RIPEMD Page](https://homes.esat.kuleuven.be/~bosselae/ripemd160.html)